In [15]:
!pip -q install geopandas shapely rasterio pyproj tqdm

In [16]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/CASA0004_Cycling/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BoroughShp
city_of_london_bike_accessible_roads.geojson
COL_cycleavailable.geojson
COL_cyclelane.geojson
Env
greater_london_bike_accessible_roads.geojson
GreatLondonShp
Roads
Slope


In [17]:
import math
import numpy as np
import rasterio
from shapely.geometry import LineString, MultiLineString
from tqdm import tqdm

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.strtree import STRtree
import numpy as np
import matplotlib.pyplot as plt

In [18]:
# —— 可调参数 ——
SLOPE_TIF = "/content/drive/MyDrive/CASA0004_Cycling/data/Slope/slope_test/slope.tif"
INTERVAL_M = 5.0             # 采样间距（米）
STAT_KIND  = "q3"             # "q3" | "mean" | "max"
SLOPE_UNIT = "degree"         # 栅格单位："degree" or "percent"
CHUNK_SIZE = 20000            # 分块行数，视内存调整
OUTPUT     = "/content/drive/MyDrive/CASA0004_Cycling/data/Roads/export_with_slope.geojson"

In [19]:
road_gdf = gpd.read_file("/content/drive/MyDrive/CASA0004_Cycling/data/Roads/export_converted1.geojson")
print(road_gdf.columns)

Index(['id', '@id', 'access', 'alt_name', 'area', 'bicycle',
       'bicycle:lanes:forward', 'bicycle:signed', 'bridge', 'bus',
       ...
       'was:cycleway:left', 'was:cycleway:right', 'was:cycleway:right:oneway',
       'was:highway', 'was:oneway', 'width', 'wikidata', 'wikimedia_commons',
       'wikipedia', 'geometry'],
      dtype='object', length=176)


In [20]:
def line_points_every(line: LineString, step: float):
    """沿 LineString 每 step 米取点（含终点）。"""
    if not line or line.length == 0:
        return []
    n = max(1, int(math.floor(line.length / step)))
    dists = [i * step for i in range(n)] + [line.length]
    return [line.interpolate(d).coords[0] for d in dists]

def geom_points(geom, step: float):
    if geom is None:
        return []
    if isinstance(geom, LineString):
        return line_points_every(geom, step)
    if isinstance(geom, MultiLineString):
        pts = []
        for part in geom.geoms:
            pts.extend(line_points_every(part, step))
        return pts
    return []

def calc_stat(vals, kind="q3"):
    if not vals:
        return 0.0
    arr = np.asarray(vals, dtype=float)
    if kind == "mean": return float(np.nanmean(arr))
    if kind == "max":  return float(np.nanmax(arr))
    return float(np.nanpercentile(arr, 75))  # q3

def slope_to_percent(v, unit="degree"):
    if unit == "percent": return float(v)
    return float(math.tan(math.radians(v)) * 100.0)

def slope_to_factor(slope_pct: float) -> float:
    # 你之前的规则
    if slope_pct <= 2:   return 1.0
    if slope_pct <= 4:   return 0.9
    if slope_pct <= 6:   return 0.75
    if slope_pct <= 8:   return 0.6
    if slope_pct <= 10:  return 0.45
    return 0.3

In [21]:
# 打开坡度栅格
ds = rasterio.open(SLOPE_TIF)
raster_crs = ds.crs
nodata = ds.nodata

# 确保道路与栅格同一 CRS
if road_gdf.crs is None:
    raise RuntimeError("road_gdf 没有 CRS，请先设定。")
if raster_crs and road_gdf.crs.to_wkt() != raster_crs.to_wkt():
    road_gdf = road_gdf.to_crs(raster_crs)

# 建字段
if "proc_slope" not in road_gdf.columns:
    road_gdf["proc_slope"] = np.nan
if "fac_3" not in road_gdf.columns:
    road_gdf["fac_3"] = np.nan


In [22]:
total = len(road_gdf)
pbar = tqdm(total=total, desc="Sampling slope", unit="feat")

start = 0
while start < total:
    end = min(start + CHUNK_SIZE, total)
    sub = road_gdf.iloc[start:end].copy()

    proc_vals = np.zeros(len(sub))
    fac_vals  = np.ones(len(sub))

    # 为每条线生成采样点 → 栅格采样 → 统计
    for i, geom in enumerate(sub.geometry.values):
        pts = geom_points(geom, INTERVAL_M)
        if not pts:
            proc_vals[i] = 0.0
            fac_vals[i]  = 1.0
            continue

        # rasterio.sample 需要 [(x,y), ...]
        smps = list(ds.sample(pts))
        vals = []
        for s in smps:
            if s is None or len(s) == 0:
                continue
            v = s[0]
            if v is None:
                continue
            if (nodata is not None and v == nodata) or np.isnan(v):
                continue
            vals.append(float(v))

        stat_v = calc_stat(vals, STAT_KIND)
        slope_pct = slope_to_percent(stat_v, SLOPE_UNIT)
        factor = slope_to_factor(slope_pct)

        proc_vals[i] = round(slope_pct, 2)
        fac_vals[i]  = round(factor, 2)

    # 写回
    road_gdf.loc[sub.index, "proc_slope"] = proc_vals
    road_gdf.loc[sub.index, "fac_3"] = fac_vals

    pbar.update(len(sub))
    start = end

pbar.close()


Sampling slope: 100%|██████████| 1360/1360 [00:01<00:00, 905.82feat/s]


In [23]:
ext = OUTPUT.lower().split(".")[-1]
if ext == "gpkg":
    road_gdf.to_file(OUTPUT, driver="GPKG")
else:
    road_gdf.to_file(OUTPUT, driver="GeoJSON")
print("Saved to:", OUTPUT)


Saved to: /content/drive/MyDrive/CASA0004_Cycling/data/Roads/export_with_slope.geojson


In [24]:
print(road_gdf.columns)

Index(['id', '@id', 'access', 'alt_name', 'area', 'bicycle',
       'bicycle:lanes:forward', 'bicycle:signed', 'bridge', 'bus',
       ...
       'was:cycleway:right:oneway', 'was:highway', 'was:oneway', 'width',
       'wikidata', 'wikimedia_commons', 'wikipedia', 'geometry', 'proc_slope',
       'fac_3'],
      dtype='object', length=178)


In [25]:
print("CRS:", ds.crs)
print("Width (cols):", ds.width)
print("Height (rows):", ds.height)
print("Transform:", ds.transform)
print("Pixel size (res):", ds.res)


CRS: EPSG:27700
Width (cols): 11676
Height (rows): 9016
Transform: | 5.00, 0.00, 503573.42|
| 0.00,-5.00, 200936.64|
| 0.00, 0.00, 1.00|
Pixel size (res): (5.000138720452213, 5.000138720452213)
